__<font color='red'>
Note: This feature is currently under development so it will not be fully functional yet. 
Also, the APIs here are experimental and subject to change.
</font>__

## Setup and loading traces
In this demo we use 8 trace files. To run this demo notebook on your laptop
1. Clone the repo `git clone https://github.com/facebookresearch/HolisticTraceAnalysis.git`
1. [Optional and recommended] Setup a conda environment. See README for details.
1. Set the `trace_dir` parameter in the next cell to the location of `tests/data/critical_path` folder in your local `HolisticTraceAnalysis` installation.

In [9]:
from hta.trace_analysis import TraceAnalysis

# trace_dir = "~/HolisticTraceAnalysis/tests/data/vision_transformer/"
trace_dir = "~/Work/hta/HolisticTraceAnalysis2/tests/data/critical_path/simple_add/"
analyzer = TraceAnalysis(trace_dir=trace_dir)

2023-09-18 10:56:04,502 - hta - trace.py:L414 - INFO - /Users/bcoutinho/Work/hta/HolisticTraceAnalysis2/tests/data/critical_path/simple_add
2023-09-18 10:56:04,508 - hta - trace_file.py:L94 - INFO - Rank to trace file map:
{0: '/Users/bcoutinho/Work/hta/HolisticTraceAnalysis2/tests/data/critical_path/simple_add/benchmark_result_493459_1694039959_trace.json.gz'}
2023-09-18 10:56:04,510 - hta - trace.py:L560 - INFO - ranks=[0]
2023-09-18 10:56:04,519 - hta - trace.py:L142 - INFO - Parsed /Users/bcoutinho/Work/hta/HolisticTraceAnalysis2/tests/data/critical_path/simple_add/benchmark_result_493459_1694039959_trace.json.gz time = 0.01 seconds 
2023-09-18 10:56:04,563 - hta - trace.py:L696 - WARNING - ProfilerStep not found in the trace. The analysis result may not be accurate.


# Critical Path Analysis

Critical path analysis is a commonly applied technique in HPC and AI/ML optimization. It can be leveraged in two ways:
1. **Efficiency opportunities** Operations/kernels on critical path should be the target of performance analysis and optimizations. They can provide the “best bang for buck” for performance improvements.This is not limited to just CPU/GPU kernels. Delays in launching or executing CUDA kernels can constitute a significant portion of the critical path as well. This could be optimized by operator fusion and CUDA graphs for example.
2. **Simulating improvements / gains:** After identifying critical path we can estimate improvements to the execution time without actually running anything. This helps us estimate the possible gains from various performance optimization.

## Approach
In a nutshell, computing the critical path involves 1) constructing a weighted DAG connecting all the operations, 2) finding the longest path in this DAG. The challenging part is constructing the DAG here. 

**Nodes**: The Nodes in the critical path graph represent points in time. Each operator/kernel thus has two nodes viz. a begin and an end node. In case of nested operators we also link the nodes in the order they appear in the callstack. An example of stacked operators is shown below-
```
          |----------------------- Op A ----------------------|
                    |--- Op B ---|        |-- Op C--|
Critical Path Graph
         (OpA.b)----(ObB.b)------(OpB.e)--(OpC.b)---(OpC.e)--(OpA.e)
```

**Edges** in this DAG can be one of two types
1. Timing edges (weight = time): include durations for the operators/kernels as well as delays to launch operators between CPU and GPU.
1. Dependency edges (weight = 0): do not have a time component but show a dependency between operations themselves. This includes data dependencies and synchronization between CPU and GPU.

## Lightweight Critical Path
Our initial implementation of Critical Path analysis does not consider dependency between PyTorch operators, this requires combining the trace with Chakra Execution Traces and processing Tensor information.

The key idea here is to simplify the dependency analysis between PyTorch operators
- We assume that all CPU operators are dependent serially on the last operator that ran on a CPU thread. This frees us from attempting to correlate with Execution Traces.
- The operator dependency part can be added back later.




## Using this noteobook

We can demonstrate critical path analysis on most PyTorch traces. However, currently traces are missing
information regarding CUDA synchronization. 

The above is fixed in this [PR](https://github.com/pytorch/pytorch/pull/105187) but not enabled by default. Please follow the documentation in this [PR](https://github.com/pytorch/pytorch/pull/105187) to enable CUDA synchronization events to get best results from this analysis.

In [2]:
analyzer.critical_path_analysis_for_rank?

Signature:
analyzer.critical_path_analysis_for_rank(
    rank: int,
    annotation: str,
    instance_id: Optional[int],
) -> List[pandas.core.frame.DataFrame]
Docstring:
Perform critical path analysis for trace events within a rank.
We further reduce the region of interest by selecting
a trace annotation and instance id. This will
limit the analysis to events within the time range of that annoation.
For example, you can use this to limit the analysis to one iteration
by passing annotation='ProfilerStep500'.

Args:
    t (Trace): Input trace data structure.
    rank (int): rank to analyze for the critical path.
    annotation (str): a trace annotation to limit the analysis to,
        such as "ProfilerStep1200"
    instance_id (int): optionally specify which instance of the annotation
        to consider. Defaults to the first instance.

Returns:
    CPGraph object that can be used to obtain statistics and further
    visualize the critical path.

    CPGraph is also a subinstance of a

In [10]:
annotation = "[param|pytorch.model.alex_net|0|0|0|measure|forward]"
instance_id = 1

In [11]:
cp_graph = analyzer.critical_path_analysis_for_rank(
    rank = 0, annotation=annotation, instance_id=instance_id)
                                                    

2023-09-18 10:56:12,324 - hta - critical_path_analysis.py:L381 - INFO - Looking up events under 1 instance of '[param|pytorch.model.alex_net|0|0|0|measure|forward]' annotation
2023-09-18 10:56:12,339 - hta - critical_path_analysis.py:L398 - INFO - Clipped dataframe has 212 events


### Run Critical Path analysis on returned CPGraph object

In [12]:
cp_graph.critical_path()

True

## Overlay and visualize the Critical Path
1. In the output trace file search for "critical" to highlight events on the critical path.
1. Edges in the critical path graph will be shown using arrows or flow events. The category names of flow events begin with "critical_path_".

In [13]:
analyzer.overlay_critical_path_analysis_for_rank?

Signature:
analyzer.overlay_critical_path_analysis_for_rank(
    rank: int,
    critical_path_graph: hta.analyzers.critical_path_analysis.CPGraph,
    output_dir: str,
    show_all_edges: bool = False,
) -> str
Docstring:
Overlay the identified critical path on top of the trace file
for visualization.

Args:
    rank (int): rank to generate the time series for.
    critical_path_graph: Critical Path Graph object generated previously
    output_dir (str): Output director to store overlaid trace.
    show_all_edges (bool): When set this will add edge events for
        all types of edges in the critical path graph. This is useful
        for debugging the algorithm.

Returns: the overlaid_trace_file path.
File:      ~/Work/hta/HolisticTraceAnalysis2/hta/trace_analysis.py
Type:      method

In [7]:
analyzer.overlay_critical_path_analysis_for_rank(
    0, cp_graph, output_dir='~/Work/hta/critical_path/simple_add/overlaid', show_all_edges=True)

2023-09-18 10:55:57,772 - hta - trace.py:L142 - INFO - Parsed /Users/bcoutinho/Work/hta/HolisticTraceAnalysis2/tests/data/critical_path/simple_add/benchmark_result_493459_1694039959_trace.json.gz time = 0.01 seconds 


'/Users/bcoutinho/Work/hta/critical_path/simple_add/overlaid/overlaid_benchmark_result_493459_1694039959_trace.json.gz'

## More details on the CPGrah object

In [16]:
from hta.analyzers.critical_path_analysis import CPGraph
CPGraph?

Init signature: CPGraph(t: 'Trace', rank: int) -> None
Docstring:     
Critical path analysis graph representation for trace from one rank.
This object constructs a graph that can be analyzed using networkx library.

We maintain a mapping between node ids -> CPNode objects
and use the integer as a node in the networkx graph datastructure.
Edges are directly used as the type is hashable.

Attributes:
    trace_df (pd.DataFrame) : dataframe of trace events used to construct this graph.
    symbol_table (TraceSymbolTable) : a symbol table used to encode the symbols in the trace.
    node_list (List[int]) : list of critical path node objects, index in this list is always the node id..
    critical_path_nodes (List[int]) : list of node ids on the critical path.
    critical_path_nodes_set (Set[int]) : set of node ids on the critical path.
    critical_path_events_set (Set[int]) : set of event ids corresponding to the critical path nodes.
Init docstring:
Initialize a graph with edges, name, 